In [2]:
from os import remove
from py7zr import SevenZipFile
import pandas as pd
import wget
import numpy as np

## Dicionário de Dados
Admitidos/Desligados: Contratação (1), Demissão(2)

Sexo: Masculino (1), Feminino (2)

Classificação de Setores CNAE 2.0: https://docs.google.com/spreadsheets/d/1bMFKpREb4YlW2ZzW1WVLdMN59ol-iLw5/export?format=xlsx

Códigos IBGE por Estado: https://www.oobj.com.br/bc/article/quais-os-c%C3%B3digos-de-cada-uf-no-brasil-465.html

In [4]:
excel = pd.ExcelFile("data/dicionario_dados.xlsx")
get_dict =  lambda x: pd.read_excel(excel, sheet_name=x)

data_dict = {
    sheet: {row[1]: row[2] for row in 
     get_dict(sheet).itertuples()}
    for sheet in excel.sheet_names[1:]
}

### Grandes grupos

In [16]:
big_group = {
	'Administração Pública, Defesa e Seguridade Social': 'Serviços',
	'Agricultura, Pecuária, Produção Florestal, Pesca e AqÜIcultura': 'Agropecuária',
	'Alojamento e Alimentação': 'Serviços',
	'Artes, Cultura, Esporte e Recreação': 'Serviços',
	'Atividades Administrativas e Serviços Complementares': 'Serviços',
	'Atividades Financeiras, de Seguros e Serviços Relacionados': 'Serviços',
	'Atividades Imobiliárias': 'Serviços',
	'Atividades Profissionais, Científicas e Técnicas': 'Serviços',
	'Comércio, Reparação de Veículos Automotores e Motocicletas': 'Comércio',
	'Construção': 'Construção',
	'Educação': 'Serviços',
	'Eletricidade e Gás': 'Indústria',
	'Indústrias Extrativas': 'Indústria',
	'Indústrias de Transformação': 'Indústria',
	'Informação e Comunicação': 'Serviços',
	'Não identificado': 'Não Identificado',
	'Organismos Internacionais e Outras Instituições Extraterritoriais': 'Serviços',
	'Outras Atividades de Serviços': 'Serviços',
	'Saúde Humana e Serviços Sociais': 'Serviços',
	'Serviços Domésticos': 'Serviços',
	'Transporte, Armazenagem e Correio': 'Serviços',
	'Água, Esgoto, Atividades de Gestão de Resíduos e Descontaminação':
	'Indústria'
}

## Leitura e processamento

In [5]:
url = lambda year, month: f"ftp://ftp.mtps.gov.br/pdet/microdados/NOVO CAGED/{year}/{year}{month:02d}/CAGEDMOV{year}{month:02d}.7z"

In [6]:
#Baixando e extraindo microdados
dfs = []
start_year = 2023
start_month = 1
dates = []
# dates = data["competênciamov"].unique()


for year in range(start_year, 2025):
    for month in range(start_month, 13):
        if f"{year}-{month:02d}-01" in dates:
            continue
        try:
            print(f"{month:02d}/{year}")
            wget.download(url(year, month), 'caged.7z')
            archive = SevenZipFile('caged.7z', mode = 'r')
            print('Microdados baixados com sucesso, passando para leitura')
            for name, fd in archive.read(name for name in archive.getnames() if name.endswith(".txt")).items():
                caged_raw = pd.read_csv(fd, delimiter=";", decimal=",")
                caged_raw = caged_raw.loc[caged_raw["uf"] == 25, :].reset_index(drop=True)
                for col in caged_raw.columns:
                    if col in data_dict:
                        caged_raw[f"{col}_cod"] = caged_raw[col]
                        caged_raw[col] = caged_raw[col].apply(lambda x: data_dict[col][x] 
                                                              if x in data_dict[col] else np.nan)

            dfs.append(caged_raw)
            archive.close()
            remove('caged.7z')
            print('Leitura concluida com sucesso')
        except:
            print('Os microdados do mês selecionado ainda não estão disponíveis')
            break

01/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
02/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
03/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
04/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
05/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
06/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
07/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
08/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
09/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
10/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
11/2023
Microdados baixados com sucesso, passando para leitura
Leitura

In [7]:
data = pd.concat(dfs)

In [8]:
data.to_csv("data/caged_pb.csv")

In [9]:
data = pd.read_csv("data/caged_pb.csv", index_col=0)
data["competênciamov"] = pd.to_datetime(data["competênciamov"], format="%Y%m")

In [10]:
data.head(1)

competênciamov    região       uf       município  \
0     2023-01-01  Nordeste  Paraíba  Pb-Joao Pessoa   

                                               seção  \
0  Comércio, Reparação de Veículos Automotores e ...   

                                           subclasse  saldomovimentação  \
0  Comércio Varejista Especializado de Equipament...                 -1   

                    cbo2002ocupação  \
0  Promotor de Vendas Especializado   

                                           categoria graudeinstrução  ...  \
0  Empregado - Geral, inclusive o empregado públi...  Médio Completo  ...   

   tipoestabelecimento_cod  tipomovimentação_cod tipodedeficiência_cod  \
0                        1                    31                     0   

  indtrabintermitente_cod indtrabparcial_cod tamestabjan_cod  \
0                       0                  0               4   

  indicadoraprendiz_cod origemdainformação_cod indicadordeforadoprazo_cod  \
0                     0                      1                          0   

  unidadesaláriocódigo_cod  
0                        5  

[1 rows x 49 columns]

In [11]:
data.groupby(["competênciamov", "saldomovimentação"], 
             as_index=False).count()[["competênciamov", "saldomovimentação", "idade"]]

competênciamov  saldomovimentação  idade
0      2023-01-01                 -1  16251
1      2023-01-01                  1  14528
2      2023-02-01                 -1  14485
3      2023-02-01                  1  15000
4      2023-03-01                 -1  18040
5      2023-03-01                  1  17226
6      2023-04-01                 -1  17140
7      2023-04-01                  1  13960
8      2023-05-01                 -1  14366
9      2023-05-01                  1  17235
10     2023-06-01                 -1  14830
11     2023-06-01                  1  14608
12     2023-07-01                 -1  13970
13     2023-07-01                  1  17447
14     2023-08-01                 -1  14502
15     2023-08-01                  1  23284
16     2023-09-01                 -1  13573
17     2023-09-01                  1  17766
18     2023-10-01                 -1  13028
19     2023-10-01                  1  16801
20     2023-11-01                 -1  12819
21     2023-11-01                  1  16477
22     2023-12-01                 -1  13457
23     2023-12-01                  1  11953
24     2024-01-01                 -1  16906
25     2024-01-01                  1  17239
26     2024-02-01                 -1  17875
27     2024-02-01                  1  17867
28     2024-03-01                 -1  18623
29     2024-03-01                  1  18886
30     2024-04-01                 -1  18189
31     2024-04-01                  1  18930

In [12]:
data = pd.read_csv("data/caged_pb.csv", index_col=0, low_memory=False)
data["competênciamov"] = data["competênciamov"].apply(lambda x: f"{str(x)[:4]}-{str(x)[4:]}-01" if "-" not in str(x) else str(x))

g = data.groupby(["competênciamov", "município", "seção", "saldomovimentação", "sexo"], as_index=False).agg({
    "graudeinstrução": "count",
})

g.to_csv("data/caged_agregado.csv")

In [13]:
g.head(1)

competênciamov       município  \
0     2023-01-01  Pb-Agua Branca   

                                               seção  saldomovimentação  \
0  Comércio, Reparação de Veículos Automotores e ...                 -1   

    sexo  graudeinstrução  
0  Homem                1

In [38]:
g = pd.read_csv("data/caged_agregado22.csv", index_col=0)
g["competênciamov"] = pd.to_datetime(g["competênciamov"])

In [39]:
g.head(1)

competênciamov       município  \
0     2023-01-01  Pb-Agua Branca   

                                               seção  saldomovimentação  \
0  Comércio, Reparação de Veículos Automotores e ...                 -1   

    sexo  graudeinstrução  
0  Homem                1

In [40]:
g2 = g.astype({"competênciamov": str}).groupby(["competênciamov",
                                                            "saldomovimentação"], as_index=False).sum()
g2["graudeinstrução"] = g2["graudeinstrução"] * g2["saldomovimentação"]

g2 = g2.groupby("competênciamov").sum()


In [41]:
g2["graudeinstrução"] = g2["graudeinstrução"] * g2["saldomovimentação"]

g2 = g2.groupby("competênciamov").sum()


In [42]:
g2 = g.astype({"competênciamov": str}).groupby(["competênciamov",
                                                            "saldomovimentação"], as_index=False).sum()
g2["graudeinstrução"] = g2["graudeinstrução"] * g2["saldomovimentação"]
g2.groupby("competênciamov").sum()["graudeinstrução"]


competênciamov
2023-01-01   -1723
2023-02-01     514
2023-03-01    -815
2023-04-01   -3181
2023-05-01    2868
2023-06-01    -223
2023-07-01    3477
2023-08-01    8782
2023-09-01    4193
2023-10-01    3773
2023-11-01    3657
2023-12-01   -1504
2024-01-01     332
2024-02-01      -9
2024-03-01     263
Name: graudeinstrução, dtype: int64

In [43]:
g2.groupby("competênciamov").sum()["graudeinstrução"].to_dict()

{'2023-01-01': -1723,
 '2023-02-01': 514,
 '2023-03-01': -815,
 '2023-04-01': -3181,
 '2023-05-01': 2868,
 '2023-06-01': -223,
 '2023-07-01': 3477,
 '2023-08-01': 8782,
 '2023-09-01': 4193,
 '2023-10-01': 3773,
 '2023-11-01': 3657,
 '2023-12-01': -1504,
 '2024-01-01': 332,
 '2024-02-01': -9,
 '2024-03-01': 263}

In [9]:
data = pd.read_csv("data/caged_agregado.csv", index_col=0)
# data["competênciamov"] = pd.to_datetime(data["competênciamov"])
data["Grupo"] = data["seção"].apply(lambda x: big_group[x] if x in big_group else np.nan)

sectors_stats = data.groupby([
     "competênciamov", "saldomovimentação","Grupo"]).sum().to_dict()["graudeinstrução"]

# # final_stats = sectors_stats.groupby(["saldomovimentação", "Grupo"]).sum()["graudeinstrução"].to_dict()


In [50]:
data.head(1)

competênciamov       município  \
0     2023-01-01  Pb-Agua Branca   

                                               seção  saldomovimentação  \
0  Comércio, Reparação de Veículos Automotores e ...                 -1   

    sexo  graudeinstrução     Grupo  
0  Homem                1  Comércio

In [10]:
sectors_stats

{('2023-01-01', -1, 'Agropecuária'): 617,
 ('2023-01-01', -1, 'Comércio'): 4399,
 ('2023-01-01', -1, 'Construção'): 2179,
 ('2023-01-01', -1, 'Indústria'): 2835,
 ('2023-01-01', -1, 'Serviços'): 6221,
 ('2023-01-01', 1, 'Agropecuária'): 290,
 ('2023-01-01', 1, 'Comércio'): 3858,
 ('2023-01-01', 1, 'Construção'): 2417,
 ('2023-01-01', 1, 'Indústria'): 1427,
 ('2023-01-01', 1, 'Serviços'): 6536,
 ('2023-02-01', -1, 'Agropecuária'): 439,
 ('2023-02-01', -1, 'Comércio'): 3835,
 ('2023-02-01', -1, 'Construção'): 2373,
 ('2023-02-01', -1, 'Indústria'): 2211,
 ('2023-02-01', -1, 'Serviços'): 5628,
 ('2023-02-01', 1, 'Agropecuária'): 160,
 ('2023-02-01', 1, 'Comércio'): 3695,
 ('2023-02-01', 1, 'Construção'): 2897,
 ('2023-02-01', 1, 'Indústria'): 1840,
 ('2023-02-01', 1, 'Serviços'): 6408,
 ('2023-03-01', -1, 'Agropecuária'): 1196,
 ('2023-03-01', -1, 'Comércio'): 4017,
 ('2023-03-01', -1, 'Construção'): 2563,
 ('2023-03-01', -1, 'Indústria'): 4067,
 ('2023-03-01', -1, 'Serviços'): 6198,
 ('2

In [11]:
sectors_stats

{('2023-01-01', -1, 'Agropecuária'): 617,
 ('2023-01-01', -1, 'Comércio'): 4399,
 ('2023-01-01', -1, 'Construção'): 2179,
 ('2023-01-01', -1, 'Indústria'): 2835,
 ('2023-01-01', -1, 'Serviços'): 6221,
 ('2023-01-01', 1, 'Agropecuária'): 290,
 ('2023-01-01', 1, 'Comércio'): 3858,
 ('2023-01-01', 1, 'Construção'): 2417,
 ('2023-01-01', 1, 'Indústria'): 1427,
 ('2023-01-01', 1, 'Serviços'): 6536,
 ('2023-02-01', -1, 'Agropecuária'): 439,
 ('2023-02-01', -1, 'Comércio'): 3835,
 ('2023-02-01', -1, 'Construção'): 2373,
 ('2023-02-01', -1, 'Indústria'): 2211,
 ('2023-02-01', -1, 'Serviços'): 5628,
 ('2023-02-01', 1, 'Agropecuária'): 160,
 ('2023-02-01', 1, 'Comércio'): 3695,
 ('2023-02-01', 1, 'Construção'): 2897,
 ('2023-02-01', 1, 'Indústria'): 1840,
 ('2023-02-01', 1, 'Serviços'): 6408,
 ('2023-03-01', -1, 'Agropecuária'): 1196,
 ('2023-03-01', -1, 'Comércio'): 4017,
 ('2023-03-01', -1, 'Construção'): 2563,
 ('2023-03-01', -1, 'Indústria'): 4067,
 ('2023-03-01', -1, 'Serviços'): 6198,
 ('2

In [17]:
import pandas as pd
import numpy as np

data = pd.read_csv("data/caged_agregado.csv", index_col=0)
# data["competênciamov"] = pd.to_datetime(data["competênciamov"])
data["Grupo"] = data["seção"].apply(lambda x: big_group[x] if x in big_group else np.nan)

sectors_stats = data.groupby([
     "competênciamov", "saldomovimentação","Grupo"]).sum().to_dict()["graudeinstrução"]

sectors_stats




{('2023-01-01', -1, 'Agropecuária'): 617,
 ('2023-01-01', -1, 'Comércio'): 4399,
 ('2023-01-01', -1, 'Construção'): 2179,
 ('2023-01-01', -1, 'Indústria'): 2835,
 ('2023-01-01', -1, 'Serviços'): 6221,
 ('2023-01-01', 1, 'Agropecuária'): 290,
 ('2023-01-01', 1, 'Comércio'): 3858,
 ('2023-01-01', 1, 'Construção'): 2417,
 ('2023-01-01', 1, 'Indústria'): 1427,
 ('2023-01-01', 1, 'Serviços'): 6536,
 ('2023-02-01', -1, 'Agropecuária'): 439,
 ('2023-02-01', -1, 'Comércio'): 3835,
 ('2023-02-01', -1, 'Construção'): 2373,
 ('2023-02-01', -1, 'Indústria'): 2211,
 ('2023-02-01', -1, 'Serviços'): 5628,
 ('2023-02-01', 1, 'Agropecuária'): 160,
 ('2023-02-01', 1, 'Comércio'): 3695,
 ('2023-02-01', 1, 'Construção'): 2897,
 ('2023-02-01', 1, 'Indústria'): 1840,
 ('2023-02-01', 1, 'Serviços'): 6408,
 ('2023-03-01', -1, 'Agropecuária'): 1196,
 ('2023-03-01', -1, 'Comércio'): 4017,
 ('2023-03-01', -1, 'Construção'): 2563,
 ('2023-03-01', -1, 'Indústria'): 4067,
 ('2023-03-01', -1, 'Serviços'): 6198,
 ('2

In [77]:
# Novo dicionário com as chaves da segunda e terceira tupla, e os valores associados
new_dict = {(key[1], key[2]): val for key, val in sectors_stats.items()}

# Exibir o novo dicionário
new_dict

# Novo dicionário com as chaves da segunda e terceira tupla, e os valores associados
new_dict = {(key[1], key[2]): val for key, val in sectors_stats.items()}

# Exibir o novo dicionário
print(new_dict)



{(-1, 'Agropecuária'): 2443,
 (-1, 'Comércio'): 4182,
 (-1, 'Construção'): 2901,
 (-1, 'Indústria'): 2530,
 (-1, 'Serviços'): 6567,
 (1, 'Agropecuária'): 207,
 (1, 'Comércio'): 5006,
 (1, 'Construção'): 3546,
 (1, 'Indústria'): 2002,
 (1, 'Serviços'): 8125,
 (-1, 'Não Identificado'): 2}

In [24]:
sectors_stats = data.groupby([
    "competênciamov", "saldomovimentação","Grupo"]).sum().to_dict()["graudeinstrução"]


In [25]:
sectors_stats


{('2023-01-01', -1, 'Agropecuária'): 617,
 ('2023-01-01', -1, 'Comércio'): 4399,
 ('2023-01-01', -1, 'Construção'): 2179,
 ('2023-01-01', -1, 'Indústria'): 2835,
 ('2023-01-01', -1, 'Serviços'): 6221,
 ('2023-01-01', 1, 'Agropecuária'): 290,
 ('2023-01-01', 1, 'Comércio'): 3858,
 ('2023-01-01', 1, 'Construção'): 2417,
 ('2023-01-01', 1, 'Indústria'): 1427,
 ('2023-01-01', 1, 'Serviços'): 6536,
 ('2023-02-01', -1, 'Agropecuária'): 439,
 ('2023-02-01', -1, 'Comércio'): 3835,
 ('2023-02-01', -1, 'Construção'): 2373,
 ('2023-02-01', -1, 'Indústria'): 2211,
 ('2023-02-01', -1, 'Serviços'): 5628,
 ('2023-02-01', 1, 'Agropecuária'): 160,
 ('2023-02-01', 1, 'Comércio'): 3695,
 ('2023-02-01', 1, 'Construção'): 2897,
 ('2023-02-01', 1, 'Indústria'): 1840,
 ('2023-02-01', 1, 'Serviços'): 6408,
 ('2023-03-01', -1, 'Agropecuária'): 1196,
 ('2023-03-01', -1, 'Comércio'): 4017,
 ('2023-03-01', -1, 'Construção'): 2563,
 ('2023-03-01', -1, 'Indústria'): 4067,
 ('2023-03-01', -1, 'Serviços'): 6198,
 ('2

In [33]:
# {key[2]: val for key, val in sectors_stats.items() if key[1] == 1}

# {key[2]: val for key, val in sectors_stats.items() if key[1] == -1}

In [34]:
sectors_stats = {
('2023-01-01', -1, 'Agropecuária'): 617,
 ('2023-01-01', -1, 'Comércio'): 4399,
 ('2023-01-01', -1, 'Construção'): 2179,
 ('2023-01-01', -1, 'Indústria'): 2835,
 ('2023-01-01', -1, 'Serviços'): 6221,
 ('2023-01-01', 1, 'Agropecuária'): 290,
 ('2023-01-01', 1, 'Comércio'): 3858,
 ('2023-01-01', 1, 'Construção'): 2417,
 ('2023-01-01', 1, 'Indústria'): 1427,
 ('2023-01-01', 1, 'Serviços'): 6536,
 ('2023-02-01', -1, 'Agropecuária'): 439,
 ('2023-02-01', -1, 'Comércio'): 3835,
 ('2023-02-01', -1, 'Construção'): 2373,
 ('2023-02-01', -1, 'Indústria'): 2211,
 ('2023-02-01', -1, 'Serviços'): 5628,
 ('2023-02-01', 1, 'Agropecuária'): 160,
 ('2023-02-01', 1, 'Comércio'): 3695,
 ('2023-02-01', 1, 'Construção'): 2897,
 ('2023-02-01', 1, 'Indústria'): 1840,
 ('2023-02-01', 1, 'Serviços'): 6408,
 ('2023-03-01', -1, 'Agropecuária'): 1196,
 ('2023-03-01', -1, 'Comércio'): 4017,
 ('2023-03-01', -1, 'Construção'): 2563,
 ('2023-03-01', -1, 'Indústria'): 4067,
 ('2023-03-01', -1, 'Serviços'): 6198,
 ('2023-03-01', 1, 'Agropecuária'): 306,
 ('2023-03-01', 1, 'Comércio'): 4804,
 ('2023-03-01', 1, 'Construção'): 2921,
 ('2023-03-01', 1, 'Indústria'): 2056,
 ('2023-03-01', 1, 'Serviços'): 7139,
 ('2023-04-01', -1, 'Agropecuária'): 1663,
 ('2023-04-01', -1, 'Comércio'): 3490,
 ('2023-04-01', -1, 'Construção'): 2398,
 ('2023-04-01', -1, 'Indústria'): 4360,
 ('2023-04-01', -1, 'Serviços'): 5230,
 ('2023-04-01', 1, 'Agropecuária'): 366,
 ('2023-04-01', 1, 'Comércio'): 3712,
 ('2023-04-01', 1, 'Construção'): 2537,
 ('2023-04-01', 1, 'Indústria'): 1485,
 ('2023-04-01', 1, 'Serviços'): 5860,
 ('2023-05-01', -1, 'Agropecuária'): 404,
 ('2023-05-01', -1, 'Comércio'): 3505,
 ('2023-05-01', -1, 'Construção'): 2196,
 ('2023-05-01', -1, 'Indústria'): 2512,
 ('2023-05-01', -1, 'Serviços'): 5750,
 ('2023-05-01', 1, 'Agropecuária'): 329,
 ('2023-05-01', 1, 'Comércio'): 4581,
 ('2023-05-01', 1, 'Construção'): 3154,
 ('2023-05-01', 1, 'Indústria'): 2292,
 ('2023-05-01', 1, 'Serviços'): 6879,
 ('2023-06-01', -1, 'Agropecuária'): 232,
 ('2023-06-01', -1, 'Comércio'): 3508,
 ('2023-06-01', -1, 'Construção'): 2970,
 ('2023-06-01', -1, 'Indústria'): 2171,
 ('2023-06-01', -1, 'Não Identificado'): 1,
 ('2023-06-01', -1, 'Serviços'): 5949,
 ('2023-06-01', 1, 'Agropecuária'): 326,
 ('2023-06-01', 1, 'Comércio'): 3889,
 ('2023-06-01', 1, 'Construção'): 2753,
 ('2023-06-01', 1, 'Indústria'): 1713,
 ('2023-06-01', 1, 'Serviços'): 5927,
 ('2023-07-01', -1, 'Agropecuária'): 211,
 ('2023-07-01', -1, 'Comércio'): 3890,
 ('2023-07-01', -1, 'Construção'): 2438,
 ('2023-07-01', -1, 'Indústria'): 1743,
 ('2023-07-01', -1, 'Não Identificado'): 1,
 ('2023-07-01', -1, 'Serviços'): 5687,
 ('2023-07-01', 1, 'Agropecuária'): 353,
 ('2023-07-01', 1, 'Comércio'): 3935,
 ('2023-07-01', 1, 'Construção'): 3082,
 ('2023-07-01', 1, 'Indústria'): 3966,
 ('2023-07-01', 1, 'Serviços'): 6111,
 ('2023-08-01', -1, 'Agropecuária'): 229,
 ('2023-08-01', -1, 'Comércio'): 3893,
 ('2023-08-01', -1, 'Construção'): 2579,
 ('2023-08-01', -1, 'Indústria'): 2005,
 ('2023-08-01', -1, 'Serviços'): 5796,
 ('2023-08-01', 1, 'Agropecuária'): 2919,
 ('2023-08-01', 1, 'Comércio'): 4409,
 ('2023-08-01', 1, 'Construção'): 3374,
 ('2023-08-01', 1, 'Indústria'): 4531,
 ('2023-08-01', 1, 'Serviços'): 8051,
 ('2023-09-01', -1, 'Agropecuária'): 262,
 ('2023-09-01', -1, 'Comércio'): 3358,
 ('2023-09-01', -1, 'Construção'): 2635,
 ('2023-09-01', -1, 'Indústria'): 1504,
 ('2023-09-01', -1, 'Não Identificado'): 2,
 ('2023-09-01', -1, 'Serviços'): 5812,
 ('2023-09-01', 1, 'Agropecuária'): 844,
 ('2023-09-01', 1, 'Comércio'): 4301,
 ('2023-09-01', 1, 'Construção'): 3042,
 ('2023-09-01', 1, 'Indústria'): 1997,
 ('2023-09-01', 1, 'Serviços'): 7582,
 ('2023-10-01', -1, 'Agropecuária'): 249,
 ('2023-10-01', -1, 'Comércio'): 3582,
 ('2023-10-01', -1, 'Construção'): 2356,
 ('2023-10-01', -1, 'Indústria'): 1526,
 ('2023-10-01', -1, 'Serviços'): 5315,
 ('2023-10-01', 1, 'Agropecuária'): 501,
 ('2023-10-01', 1, 'Comércio'): 4617,
 ('2023-10-01', 1, 'Construção'): 3302,
 ('2023-10-01', 1, 'Indústria'): 1867,
 ('2023-10-01', 1, 'Serviços'): 6514,
 ('2023-11-01', -1, 'Agropecuária'): 263,
 ('2023-11-01', -1, 'Comércio'): 3365,
 ('2023-11-01', -1, 'Construção'): 2632,
 ('2023-11-01', -1, 'Indústria'): 1453,
 ('2023-11-01', -1, 'Serviços'): 5107,
 ('2023-11-01', 1, 'Agropecuária'): 207,
 ('2023-11-01', 1, 'Comércio'): 4591,
 ('2023-11-01', 1, 'Construção'): 3021,
 ('2023-11-01', 1, 'Indústria'): 1582,
 ('2023-11-01', 1, 'Serviços'): 7076,
 ('2023-12-01', -1, 'Agropecuária'): 319,
 ('2023-12-01', -1, 'Comércio'): 3368,
 ('2023-12-01', -1, 'Construção'): 1773,
 ('2023-12-01', -1, 'Indústria'): 1690,
 ('2023-12-01', -1, 'Serviços'): 6308,
 ('2023-12-01', 1, 'Agropecuária'): 162,
 ('2023-12-01', 1, 'Comércio'): 3723,
 ('2023-12-01', 1, 'Construção'): 1835,
 ('2023-12-01', 1, 'Indústria'): 1120,
 ('2023-12-01', 1, 'Serviços'): 5114,
 ('2024-01-01', -1, 'Agropecuária'): 649,
 ('2024-01-01', -1, 'Comércio'): 4657,
 ('2024-01-01', -1, 'Construção'): 2588,
 ('2024-01-01', -1, 'Indústria'): 2017,
 ('2024-01-01', -1, 'Serviços'): 6996,
 ('2024-01-01', 1, 'Agropecuária'): 339,
 ('2024-01-01', 1, 'Comércio'): 4001,
 ('2024-01-01', 1, 'Construção'): 3574,
 ('2024-01-01', 1, 'Indústria'): 1698,
 ('2024-01-01', 1, 'Serviços'): 7627,
 ('2024-02-01', -1, 'Agropecuária'): 1123,
 ('2024-02-01', -1, 'Comércio'): 4271,
 ('2024-02-01', -1, 'Construção'): 3009,
 ('2024-02-01', -1, 'Indústria'): 3209,
 ('2024-02-01', -1, 'Serviços'): 6264,
 ('2024-02-01', 1, 'Agropecuária'): 231,
 ('2024-02-01', 1, 'Comércio'): 4160,
 ('2024-02-01', 1, 'Construção'): 3662,
 ('2024-02-01', 1, 'Indústria'): 1911,
 ('2024-02-01', 1, 'Serviços'): 7903,
 ('2024-03-01', -1, 'Agropecuária'): 2443,
 ('2024-03-01', -1, 'Comércio'): 4182,
 ('2024-03-01', -1, 'Construção'): 2901,
 ('2024-03-01', -1, 'Indústria'): 2530,
 ('2024-03-01', -1, 'Serviços'): 6567,
 ('2024-03-01', 1, 'Agropecuária'): 207,
 ('2024-03-01', 1, 'Comércio'): 5006,
 ('2024-03-01', 1, 'Construção'): 3546,
 ('2024-03-01', 1, 'Indústria'): 2002,
 ('2024-03-01', 1, 'Serviços'): 8125,
}

In [37]:
selected_date: str = "2023-02-01"
filtered_data = {key[2]: val for key, val in sectors_stats.items() if key[0] == selected_date and key[1] == 1}

In [38]:
filtered_data

{'Agropecuária': 160,
 'Comércio': 3695,
 'Construção': 2897,
 'Indústria': 1840,
 'Serviços': 6408}